<a href="https://colab.research.google.com/github/Akshatasthana18/Celebal25/blob/main/loan_Approval_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sonalisingh1411/loan-approval-prediction")

print("Path to dataset files:", path)

100%|██████████| 13.6k/13.6k [00:00<00:00, 20.6MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sonalisingh1411/loan-approval-prediction/versions/1


In [3]:
!pip install pandas faiss-cpu sentence-transformers openai transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [9]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
from transformers import pipeline
import os

# Load dataset
# The path from kagglehub.dataset_download is a directory, not a file.
# We need to find the actual CSV file within this directory.
dataset_dir = "/root/.cache/kagglehub/datasets/sonalisingh1411/loan-approval-prediction/versions/1"
csv_file_path = os.path.join(dataset_dir, "Training Dataset.csv") # Using the correct CSV file name

df = pd.read_csv(csv_file_path)
df.fillna("NA", inplace=True)

# Convert rows into readable text for retrieval
def row_to_text(row):
    return " | ".join(f"{col}: {row[col]}" for col in df.columns)

documents = [row_to_text(row) for _, row in df.iterrows()]

# Step 1: Embed Documents
embed_model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and fast
doc_embeddings = embed_model.encode(documents, convert_to_numpy=True)

# Step 2: Create FAISS Index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# Step 3: Retrieval Function
def retrieve_docs(query, top_k=5):
    query_vec = embed_model.encode([query])
    distances, indices = index.search(query_vec, top_k)
    return [documents[i] for i in indices[0]]

# Step 4: Generate Response
def generate_answer(query, context):
    try:
        # Replace with your OpenAI API key
        openai.api_key = "your-openai-key"
        response = openai.ChatCompletion.create(
            model="gpt-4",  # or gpt-3.5-turbo
            messages=[
                {"role": "system", "content": "You are a helpful assistant for loan-related questions."},
                {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"}
            ],
            temperature=0.5,
            max_tokens=300,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print("OpenAI failed:", e)
        # fallback using Hugging Face if OpenAI fails
        fallback_pipe = pipeline("text-generation", model="google/flan-t5-small")
        return fallback_pipe(f"Context: {context}\n\nQuestion: {query}", max_length=300)[0]['generated_text']

# Step 5: Ask a Question
def ask_bot(question):
    context_docs = retrieve_docs(question)
    context = "\n".join(context_docs)
    answer = generate_answer(question, context)
    return answer

# Example
if __name__ == "__main__":
    while True:
        user_q = input("\nAsk your loan-related question (or type 'exit'): ")
        if user_q.lower() == "exit":
            break
        print("\nBot:", ask_bot(user_q))

/tmp/ipython-input-9-3928595618.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("NA", inplace=True)



Ask your loan-related question (or type 'exit'): How much time I have?
OpenAI failed: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'Gemma3nForConditionalGeneration', 'Gemma3nForCausa


Bot: Context: Loan_ID: LP001273 | Gender: Male | Married: Yes | Dependents: 0 | Education: Graduate | Self_Employed: No | ApplicantIncome: 6000 | CoapplicantIncome: 2250.0 | LoanAmount: 265.0 | Loan_Amount_Term: 360.0 | Credit_History: NA | Property_Area: Semiurban | Loan_Status: N
Loan_ID: LP002050 | Gender: Male | Married: Yes | Dependents: 1 | Education: Graduate | Self_Employed: Yes | ApplicantIncome: 10000 | CoapplicantIncome: 0.0 | LoanAmount: 155.0 | Loan_Amount_Term: 360.0 | Credit_History: 1.0 | Property_Area: Rural | Loan_Status: N
Loan_ID: LP001722 | Gender: Male | Married: Yes | Dependents: 0 | Education: Graduate | Self_Employed: No | ApplicantIncome: 150 | CoapplicantIncome: 1800.0 | LoanAmount: 135.0 | Loan_Amount_Term: 360.0 | Credit_History: 1.0 | Property_Area: Rural | Loan_Status: N
Loan_ID: LP002863 | Gender: Male | Married: Yes | Dependents: 3+ | Education: Graduate | Self_Employed: No | ApplicantIncome: 6406 | CoapplicantIncome: 0.0 | LoanAmount: 150.0 | Loan_Amo

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'Gemma3nForConditionalGeneration', 'Gemma3nForCausa


Bot: Context: Loan_ID: LP001267 | Gender: Female | Married: Yes | Dependents: 2 | Education: Graduate | Self_Employed: No | ApplicantIncome: 1378 | CoapplicantIncome: 1881.0 | LoanAmount: 167.0 | Loan_Amount_Term: 360.0 | Credit_History: 1.0 | Property_Area: Urban | Loan_Status: N
Loan_ID: LP001754 | Gender: Male | Married: Yes | Dependents: NA | Education: Not Graduate | Self_Employed: Yes | ApplicantIncome: 4735 | CoapplicantIncome: 0.0 | LoanAmount: 138.0 | Loan_Amount_Term: 360.0 | Credit_History: 1.0 | Property_Area: Urban | Loan_Status: N
Loan_ID: LP001673 | Gender: Male | Married: No | Dependents: 0 | Education: Graduate | Self_Employed: Yes | ApplicantIncome: 11000 | CoapplicantIncome: 0.0 | LoanAmount: 83.0 | Loan_Amount_Term: 360.0 | Credit_History: 1.0 | Property_Area: Urban | Loan_Status: N
Loan_ID: LP001688 | Gender: Male | Married: Yes | Dependents: 1 | Education: Not Graduate | Self_Employed: No | ApplicantIncome: 3500 | CoapplicantIncome: 1083.0 | LoanAmount: 135.0 | L